### Provide by

นายธนชาติ เสถียรจารุการ 63340500021 <br>

นายพชพล เพชรรัตน์ 63340500036

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import resample
from sklearn.preprocessing import QuantileTransformer
import math

RandomState = 1
df = pd.read_csv('credit_card_churn.csv')
df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 'CLIENTNUM'], axis=1)
df = df.rename(columns={'Attrition_Flag': 'y',
                         'Customer_Age': 'age',
                         'Gender': 'gender',
                         'Dependent_count': 'dependency',
                         'Education_Level': 'education',
                         'Marital_Status': 'marital',
                         'Income_Category': 'income',
                         'Card_Category': 'card',
                         'Months_on_book': 'book_period',
                         'Total_Relationship_Count': 'total_product',
                         'Months_Inactive_12_mon': 'month_inactive',
                         'Contacts_Count_12_mon': 'contact_num',
                         'Credit_Limit': 'credit_limit',
                         'Total_Revolving_Bal': 'revolving_balance',
                         'Avg_Open_To_Buy': 'open2buy',
                         'Total_Amt_Chng_Q4_Q1': 'transaction_change',
                         'Total_Trans_Amt': 'transaction_amount',
                         'Total_Trans_Ct': 'transaction_count',
                         'Total_Ct_Chng_Q4_Q1': 'transaction_count_change',
                         'Avg_Utilization_Ratio': 'utilization_ratio',
                         }
)
df['y'] = df['y'].replace(['Attrited Customer', 'Existing Customer'], [0, 1])

### Data exploration

In [ ]:
df.info()

##### Target
The total number of data is 10,127 samples devided into
- Attrited: 8,500 samples
<br>

- Existing: 1,627 samples

**the dataset is imbalanced**

In [ ]:
count = df['y'].value_counts(); print(count)
plt.figure(figsize = (4, 3))
sns.histplot(data = df, x = 'y')
plt.show()
print(f"major target: {(count[1]/(count[0] + count[1])) * 100:0.2f}%")

##### resample data

In [ ]:
major_target = df.loc[df['y'] == 1]
minor_target = df.loc[df['y'] == 0]
upsampling_df = resample(minor_target, n_samples=major_target.shape[0], replace=True, random_state=RandomState)
df = pd.concat([major_target, upsampling_df], ignore_index=True)
df = df.sample(frac = 1, ignore_index=True)

count = df['y'].value_counts(); print(count)
plt.figure(figsize = (4, 3))
sns.histplot(data = df, x = 'y')
plt.show()

print(f"major target: {(count[1]/(count[0] + count[1])) * 100:0.2f}%")

##### Explore numerical data

the data that seem to have an outlier is listed below
- age
- transaction_change
- transaction_count_change

In [ ]:
def seperateDataType(df): ## return list of numerical and categorical data
    cols = df.columns
    num_data = [i for i in cols if (len(df[i].unique()) > 7 and df[i].dtype != 'object')]
    cat_data = list(set(cols) - set(num_data))
    return num_data, cat_data

num_data, cat_data = seperateDataType(df)
plot_num = 1
plt.figure(figsize = (15,27))
for i in num_data:
    ax = plt.subplot(7, 3, plot_num)
    sns.histplot(data=df, x=i, hue='y', kde=True)
    plot_num += 1
plt.show()


- Use IQR to remove outlier

In [ ]:
def removeNumericalOutlier(df:pd.DataFrame, feature_list:list):
    new_df = df.copy()
    for feature in feature_list:       
        q1 = df[feature].quantile(0.25)
        q3 = df[feature].quantile(0.75)
        IQR = q3 - q1
        lower_bound = q1 - 1.5*IQR
        upper_bound = q3 + 1.5*IQR
        new_df = new_df[(new_df[feature]>lower_bound)&(new_df[feature]<upper_bound)]
    return new_df

outlier_list = ['age', 'transaction_change', 'transaction_count_change']
df = removeNumericalOutlier(df, outlier_list)

plot_num = 1
plt.figure(figsize = (9, 4))
for i in outlier_list:
    ax = plt.subplot(1, 3, plot_num)
    sns.histplot(data=df, x=i, hue='y')
    plot_num += 1
plt.show()

- Use quantile transformer to correct skewed data and standardlize

In [ ]:
def correctSkewed(df:pd.DataFrame, skewed_data_list:list):
    ##### correct skewed data
    quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=1)
    x_skew = df[skewed_data_list].values
    X_trans = quantile_transformer.fit_transform(x_skew)
    df[skewed_data_list] = X_trans
    return df

df = correctSkewed(df, num_data)
plot_num = 1
plt.figure(figsize = (15,27))
for i in num_data:
    ax = plt.subplot(7, 3, plot_num)
    sns.histplot(data=df, x=i, hue='y', kde=True)
    plot_num += 1
plt.show()

- plot heatmap of numerical data correlation

In [ ]:
num_df = df[num_data]
num_df['y'] = df['y']
plt.figure(figsize = (10,7))
sns.heatmap(round(num_df.corr(), 2), annot=True)
plt.show()

- visualize high correlation to target
<br>

*the distribution of some data that have a high correlation to the target seems to be the same value as listed below but the pair between Transaction_amount and transaction_count we not sure how this relation effect to model and we consider testing in the training model session*

In [ ]:
high_corr_data_list = ['transaction_count', 'transaction_count_change', 'utilization_ratio', 'transaction_amount', 'revolving_balance']
plot_num = 1
plt.figure(figsize = (15,27))
for i in high_corr_data_list:
    ax = plt.subplot(7, 3, plot_num)
    sns.boxplot(data=num_df, x='y', y=i)
    plot_num += 1
plt.show()

- visualize relation between numerical data

*the result is related to the correlation heatmap above, the pair with high correlation for each other show some mathematical function*

In [ ]:
sns.pairplot(data=num_df, hue='y')
plt.show()

##### Explore categorical data

In [ ]:
plot_num = 1
plt.figure(figsize = (15,27))
for i in cat_data:
    ax = plt.subplot(7, 3, plot_num)
    sns.histplot(data=df, x=i, hue='y')
    plot_num += 1
plt.show()

- group card category 

In [ ]:
df['card'] = df['card'].replace(['Silver', 'Gold', 'Platinum'], ['not_blue']*3)
plt.figure(figsize = (3,4))
sns.histplot(data=df, x='card', hue='y')
plt.show()

- plot heatmap of categorial data correlation

In [ ]:
cat_df = df[cat_data]
cat_df = cat_df.drop(['y'],axis=1)
cat_df = pd.get_dummies(cat_df)
cat_df['y'] = df['y']
plt.figure(figsize = (32,18))
sns.heatmap(round(cat_df.corr(), 2), annot=True)
plt.show()

- visualize high correlation to target
<br>

*the feature 'contact_num' and 'month_inactive' some labels have a high ratio of the target but the feature 'total_product' is not obviously show*

In [ ]:
high_corr_data_list = ['contact_num', 'month_inactive', 'total_product']
plot_num = 1
plt.figure(figsize = (15,27))
for i in high_corr_data_list:
    ax = plt.subplot(7, 3, plot_num)
    sns.countplot(data=cat_df, x=i, hue='y')
    plot_num += 1
plt.show()

- visualize relation between categorical data

In [ ]:
plot_num = 1
plt.figure(figsize = (60,60))
for i in cat_data:
    for j in cat_data:
        ax = plt.subplot(10, 10, plot_num)
        sns.countplot(data=df, x=i, hue=j)
        plot_num += 1
plt.show()

##### visualize relation between categoriacal and numerical data

In [ ]:
plot_num = 1
plt.figure(figsize = (60,60))
for i in cat_data:
    for j in num_data:
        ax = plt.subplot(10, 10, plot_num)
        sns.boxplot(data=df, x=i, y=j ,hue='y')
        plot_num += 1
plt.show()

##### Data exploration summary

1. upsampling data

2. use IQR to remove outlier

3. use quantile transformer to correct skewed data and standardlize

4. list of numerical features below have a high correlation to the target but when we visualize one thing we realized is some features have close value for each other
    * ['transaction_count', 'transaction_count_change', 'utilization_ratio', 'transaction_amount', 'revolving_balance']

<br>
<br\>

5. list of pair that have the relation between numerical data
    - Utilization_ratio and revolving_balance
    - Transaction_count and transaction_amount
    - Book_period and age
    - Open2buy and credit_limit

<br>
<br\>

6. list of categorical features below have high correlation to the target
    - ['contact_num', 'month_inactive', 'total_product']

### Data Preprocessing


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import resample
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import classification_report, silhouette_score

RandomState = 1

def importDataset():
    df = pd.read_csv('credit_card_churn.csv')
    df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 'CLIENTNUM'], axis=1)
    df = df.rename(columns={'Attrition_Flag': 'y',
                         'Customer_Age': 'age',
                         'Gender': 'gender',
                         'Dependent_count': 'dependency',
                         'Education_Level': 'education',
                         'Marital_Status': 'marital',
                         'Income_Category': 'income',
                         'Card_Category': 'card',
                         'Months_on_book': 'book_period',
                         'Total_Relationship_Count': 'total_product',
                         'Months_Inactive_12_mon': 'month_inactive',
                         'Contacts_Count_12_mon': 'contact_num',
                         'Credit_Limit': 'credit_limit',
                         'Total_Revolving_Bal': 'revolving_balance',
                         'Avg_Open_To_Buy': 'open2buy',
                         'Total_Amt_Chng_Q4_Q1': 'transaction_change',
                         'Total_Trans_Amt': 'transaction_amount',
                         'Total_Trans_Ct': 'transaction_count',
                         'Total_Ct_Chng_Q4_Q1': 'transaction_count_change',
                         'Avg_Utilization_Ratio': 'utilization_ratio',
                         }
    )
    df['y'] = df['y'].replace(['Attrited Customer', 'Existing Customer'], [0, 1])
    return df

def seperateDataType(df): ## return list of numerical and categorical data
    cols = df.columns
    num_data = [i for i in cols if (len(df[i].unique()) > 7 and df[i].dtype != 'object')]
    cat_data = list(set(cols) - set(num_data))
    return num_data, cat_data

def reSampling(df:pd.DataFrame, up_flag:bool, down_flag:bool):
    major_target = df.loc[df['y'] == 1]
    minor_target = df.loc[df['y'] == 0]
    new_df = df.copy()
    if up_flag:
        upsampling_df = resample(minor_target, n_samples=major_target.shape[0], replace=True, random_state=RandomState)
        new_df = pd.concat([major_target, upsampling_df], ignore_index=True)
        new_df = new_df.sample(frac = 1, ignore_index=True)
    elif down_flag:
        downsampling_df = resample(major_target, n_samples=minor_target.shape[0], replace=True, random_state=RandomState)
        new_df = pd.concat([minor_target, downsampling_df], ignore_index=True)
        new_df = new_df.sample(frac = 1, ignore_index=True)
    return new_df

def removeOutlier(df:pd.DataFrame, feature_list:list, flag:bool):
    new_df = df.copy()
    if flag:
        for feature in feature_list:
            q1 = df[feature].quantile(0.25)
            q3 = df[feature].quantile(0.75)
            IQR = q3 - q1
            lower_bound = q1 - 1.5*IQR
            upper_bound = q3 + 1.5*IQR
            new_df = new_df[(new_df[feature]>lower_bound)&(new_df[feature]<upper_bound)]
    return new_df

def correctSkewed(df:pd.DataFrame, skewed_data_list:list, flag:bool):
    new_df = df.copy()
    if flag:
        quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=1)
        x_skew = new_df[skewed_data_list].values
        X_trans = quantile_transformer.fit_transform(x_skew)
        new_df[skewed_data_list] = X_trans
    return new_df

def preprocessData(drop_list:list, 
                    select_list:list, 
                    upsampling_flag:bool = True, 
                    downsampling_flag:bool = True, 
                    remove_outlier_flag:bool = True, 
                    correct_skewed_flag:bool = True):
    df = importDataset()
    df = reSampling(df, upsampling_flag, downsampling_flag)
    outlier_list = ['age', 'transaction_change', 'transaction_count_change']
    df = removeOutlier(df, outlier_list, remove_outlier_flag)
    num_data, cat_data = seperateDataType(df)
    df = correctSkewed(df, num_data, correct_skewed_flag)
    if drop_list is not None:
        df = df.drop(drop_list, axis=1)
    elif select_list is not None:
        df = df[select_list]
    df_y = df['y']
    df_x = df.drop(['y'], axis=1)
    df_x = pd.get_dummies(df_x)
    x = df_x.values
    y = df_y.values
    scaler = StandardScaler()
    x_scale = scaler.fit_transform(x)
    return x_scale, y

### Model Construction and Analysis

In [16]:
def trainK_mean(drop_list:list, 
                    select_list:list, 
                    upsampling_flag:bool = True, 
                    downsampling_flag:bool = True, 
                    remove_outlier_flag:bool = True, 
                    correct_skewed_flag:bool = True):
    ##### get x and y data from preprocessing
    x, y = preprocessData(drop_list, select_list, upsampling_flag, downsampling_flag, remove_outlier_flag, correct_skewed_flag)
    ##### Split train and test data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)
    ##### Declare model and model parameters
    model = KMeans(n_clusters=2, random_state=1, max_iter=1000)
    parameters = {'init':('k-means++', 'random'),
                  'algorithm':("auto", "full", "elkan")}
    ##### Train model with gridsearchCV and split k-fold = 10
    clf = GridSearchCV(model, parameters, scoring='f1', verbose=3, return_train_score=True, n_jobs=-1, cv = 10)
    clf.fit(x_train, y_train)
    ##### Test model, visualize classification report and confusion matrix
    print(clf.best_params_)
    y_pred = clf.predict(x_test)
    print("K means Classification report \n=======================")
    print(classification_report(y_true=y_test, y_pred=y_pred))
    print("K means Confusion matrix \n=======================")
    return clf

def trainAgglomerative(drop_list:list, 
                    select_list:list, 
                    upsampling_flag:bool = True, 
                    downsampling_flag:bool = True, 
                    remove_outlier_flag:bool = True, 
                    correct_skewed_flag:bool = True):
    ##### get x and y data from preprocessing
    x, y = preprocessData(drop_list, select_list, upsampling_flag, downsampling_flag, remove_outlier_flag, correct_skewed_flag)
    ##### Declare model and model parameters
    model = AgglomerativeClustering(n_clusters=2, compute_distances=True)
    y_pred = model.fit_predict(x, y)
    print("Agglomerative Clustering Classification report \n=======================")
    print(classification_report(y_true=y, y_pred=y_pred))
    print("Agglomerative Clustering Confusion matrix \n=======================")
    return model

def trainDBSCAN(drop_list:list, 
                    select_list:list, 
                    upsampling_flag:bool = True, 
                    downsampling_flag:bool = True, 
                    remove_outlier_flag:bool = True, 
                    correct_skewed_flag:bool = True):
    ##### get x and y data from preprocessing
    x, y = preprocessData(drop_list, select_list, upsampling_flag, downsampling_flag, remove_outlier_flag, correct_skewed_flag)
    ##### Declare model and model parameters
    for i in range(1,100,1):
        model = DBSCAN(eps=i, n_jobs=-1)
        y_pred = model.fit_predict(x, y)
        if(max(y_pred) == 1):
            print('eps= ' + str(i))
            y_pred = model.fit_predict(x, y)
            print("Agglomerative Clustering Classification report \n=======================")
            print(classification_report(y_true=y, y_pred=y_pred))
            print("Agglomerative Clustering Confusion matrix \n=======================")
            break
    return model

##### EXP 1 preprocessing data

- raw data

In [20]:
drop_list = None
select_list = None
trainK_mean(drop_list, select_list, upsampling_flag = False, downsampling_flag = False, remove_outlier_flag = False, correct_skewed_flag = False)
trainAgglomerative(drop_list, select_list, upsampling_flag = False, downsampling_flag = False, remove_outlier_flag = False, correct_skewed_flag = False)
trainDBSCAN(drop_list, select_list, upsampling_flag = False, downsampling_flag = False, remove_outlier_flag = False, correct_skewed_flag = False)


Fitting 10 folds for each of 6 candidates, totalling 60 fits
{'algorithm': 'auto', 'init': 'k-means++'}
K means Classification report 
              precision    recall  f1-score   support

           0       0.16      0.45      0.23       331
           1       0.83      0.53      0.64      1695

    accuracy                           0.51      2026
   macro avg       0.49      0.49      0.44      2026
weighted avg       0.72      0.51      0.58      2026

K means Confusion matrix 
Agglomerative Clustering Classification report 
              precision    recall  f1-score   support

           0       0.16      0.93      0.27      1627
           1       0.84      0.07      0.13      8500

    accuracy                           0.21     10127
   macro avg       0.50      0.50      0.20     10127
weighted avg       0.73      0.21      0.15     10127

Agglomerative Clustering Confusion matrix 
Agglomerative Clustering Classification report 
              precision    recall  f1-score   

DBSCAN(eps=11, n_jobs=-1)

- upsampling

In [21]:
drop_list = None
select_list = None
trainK_mean(drop_list, select_list, upsampling_flag = True, downsampling_flag = False, remove_outlier_flag = False, correct_skewed_flag = False)
trainAgglomerative(drop_list, select_list, upsampling_flag = True, downsampling_flag = False, remove_outlier_flag = False, correct_skewed_flag = False)
trainDBSCAN(drop_list, select_list, upsampling_flag = True, downsampling_flag = False, remove_outlier_flag = False, correct_skewed_flag = False)


Fitting 10 folds for each of 6 candidates, totalling 60 fits
{'algorithm': 'auto', 'init': 'random'}
K means Classification report 
              precision    recall  f1-score   support

           0       0.53      0.57      0.55      1711
           1       0.53      0.49      0.51      1689

    accuracy                           0.53      3400
   macro avg       0.53      0.53      0.53      3400
weighted avg       0.53      0.53      0.53      3400

K means Confusion matrix 
Agglomerative Clustering Classification report 
              precision    recall  f1-score   support

           0       0.50      0.93      0.65      8500
           1       0.50      0.07      0.12      8500

    accuracy                           0.50     17000
   macro avg       0.50      0.50      0.39     17000
weighted avg       0.50      0.50      0.39     17000

Agglomerative Clustering Confusion matrix 
Agglomerative Clustering Classification report 
              precision    recall  f1-score   sup

DBSCAN(eps=11, n_jobs=-1)

- downsampling

In [22]:
drop_list = None
select_list = None
trainK_mean(drop_list, select_list, upsampling_flag = False, downsampling_flag = True, remove_outlier_flag = False, correct_skewed_flag = False)
trainAgglomerative(drop_list, select_list, upsampling_flag = False, downsampling_flag = True, remove_outlier_flag = False, correct_skewed_flag = False)
trainDBSCAN(drop_list, select_list, upsampling_flag = False, downsampling_flag = True, remove_outlier_flag = False, correct_skewed_flag = False)


Fitting 10 folds for each of 6 candidates, totalling 60 fits
{'algorithm': 'auto', 'init': 'k-means++'}
K means Classification report 
              precision    recall  f1-score   support

           0       0.49      0.41      0.45       340
           1       0.45      0.53      0.49       311

    accuracy                           0.47       651
   macro avg       0.47      0.47      0.47       651
weighted avg       0.47      0.47      0.47       651

K means Confusion matrix 
Agglomerative Clustering Classification report 
              precision    recall  f1-score   support

           0       0.50      0.93      0.65      1627
           1       0.53      0.08      0.13      1627

    accuracy                           0.50      3254
   macro avg       0.52      0.50      0.39      3254
weighted avg       0.52      0.50      0.39      3254

Agglomerative Clustering Confusion matrix 
Agglomerative Clustering Classification report 
              precision    recall  f1-score   

DBSCAN(eps=11, n_jobs=-1)

##### EXP 2 Select good feature

##### EXP 3 Cut bad feature